# Customized soil queries and input files
- feed lat/lon info of each site into soilquery.py to retrieve soil info
- store output for each location

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import seaborn as sns
import plotly.express as px

from soilquery import soilquery

### Read in site summary info

In [2]:
df_summary = pd.read_csv('/home/disk/eos8/ach315/upscale/weadata/site_summary.csv', 
                         index_col=0, dtype={'site':str})
df_siteyears = pd.read_csv('/home/disk/eos8/ach315/upscale/weadata/siteyears_filtered.csv', 
                            index_col=0, dtype={'site':str})
sites = set(df_siteyears.site)
df_sites = df_summary[df_summary.site.isin(sites)]
df_sites.head()

,site,class,station,state,tzone,lat,lon,years,area,perct_irri
6,722070,1,SAVANNAH INTL AP,GA,-5,32.117,-81.200,22,2445.923794,6.585904
7,722080,1,CHARLESTON INTL ARPT,SC,-5,32.900,-80.033,25,10052.406880,5.735219
9,722140,1,TALLAHASSEE REGIONAL AP [ISIS],FL,-5,30.400,-84.350,19,9510.127270,13.348148
10,722170,1,MACON MIDDLE GA REGIONAL AP,GA,-5,32.683,-83.650,22,2731.632292,18.039572
11,722180,1,AUGUSTA BUSH FIELD,GA,-5,33.367,-81.967,18,5940.794366,8.403471


### Query soil info for all sites

In [3]:
lats = df_sites.lat
lons = df_sites.lon
df_soils = pd.DataFrame()

for site, lat, lon in zip(sites, lats,lons):
    try:
        df_soil = soilquery(round(lat,2), round(lon,2))
        df_soil['site'] = site
        df_soil['lat'] = lat
        df_soil['lon'] = lon
        df_soils = df_soils.append(df_soil)
    except:
        print(site, 
              df_sites[df_sites.site==site].station.values[0], 
              df_sites[df_sites.site==site].state.values[0])

df_soils.dropna(axis=0, inplace=True)
df_soils.reset_index(drop=True, inplace=True)
df_soils.head()

724220 LEXINGTON BLUEGRASS AP KY
725170 ALLENTOWN LEHIGH VALLEY INTL PA
725210 AKRON AKRON-CANTON REG AP OH
723403 LITTLE ROCK ADAMS FIELD AR
723560 TULSA INTERNATIONAL AIRPORT OK
723530 OKLAHOMA CITY WILL ROGERS WOR OK
722530 SAN ANTONIO INTL AP TX
723630 AMARILLO INTERNATIONAL AP [CANYON - UT] TX
726410 MADISON DANE CO REGIONAL ARPT [ISIS] WI
725040 BRIDGEPORT SIKORSKY MEMORIAL CT
722080 CHARLESTON INTL ARPT SC
723270 NASHVILLE INTERNATIONAL AP TN
726590 ABERDEEN REGIONAL ARPT SD
724320 EVANSVILLE REGIONAL AP IN
725190 SYRACUSE HANCOCK INT'L ARPT NY
724210 CINCINNATI NORTHERN KY AP KY
722560 WACO REGIONAL AP TX


,cokey,chkey,prcent,slope_r,slope,hzname,depth,awc,clay,silt,sand,OM,dbthirdbar,th33,bd,site,lat,lon
0,20681154,60804747,60.0,1,2,H1,0.0,0.07,7.0,6.6,86.4,1.50,1.55,16.6,-0.1505,722140,32.117,-81.20
1,20681154,60804748,60.0,1,2,H2,71.0,0.11,25.0,18.0,57.0,0.25,1.63,24.9,-0.2327,722140,32.117,-81.20
2,20681154,60804749,60.0,1,2,H3,150.0,0.11,27.5,17.4,55.1,0.25,1.63,25.9,-0.2427,722140,32.117,-81.20
3,20646330,60671139,85.0,3,5,A,0.0,0.04,1.5,4.9,93.6,0.53,1.45,5.2,-0.0375,725330,30.400,-84.35
4,20646330,60671138,85.0,3,5,C,18.0,0.04,1.0,1.1,97.9,0.05,1.45,4.0,-0.0255,725330,30.400,-84.35


### Bin soil quert outputs into 5 soil depth categories

In [4]:
df_soils['depth_category'] = np.nan
df_soils.loc[(df_soils.depth > -1) & (df_soils.depth <= 0),'depth_category'] = '0'
df_soils.loc[(df_soils.depth > 0) & (df_soils.depth <= 50), 'depth_category'] = '0-50'
df_soils.loc[(df_soils.depth > 50) & (df_soils.depth <= 100), 'depth_category'] = '50-100'
df_soils.loc[(df_soils.depth > 100) & (df_soils.depth <= 150), 'depth_category'] = '100-150'
df_soils.loc[df_soils.depth >150, 'depth_category'] = '>150'

### Create soil texture lookup table

In [5]:
sand_min = [80,50,30,0,50,0,20,50,0,0]
sand_max = [100,80,50,50,80,30,50,70,20,50]
silt_min = [0,0,30,50,0,50,20,0,50,0]
silt_max = [20,50,50,100,30,80,50,20,70,50]
clay_min = [0,0,0,0,20,20,20,30,30,30]
clay_max = [20,20,20,20,30,30,30,50,50,100]
texture_class = ['sand', 'sandy loam', 'loam', 'silt loam', 'sandy clay loam', 
                 'silty clay loam', 'clay loam', 'sandy clay', 'silty clay', 'clay']

df_texture = pd.DataFrame({'sand_min': sand_min,
                           'sand_max': sand_max,
                           'silt_min': silt_min, 
                           'silt_max': silt_max,
                           'clay_min': clay_min, 
                           'clay_max': clay_max,
                           'texture': texture_class})
df_texture.head()

,sand_min,sand_max,silt_min,silt_max,clay_min,clay_max,texture
0,80,100,0,20,0,20,sand
1,50,80,0,50,0,20,sandy loam
2,30,50,30,50,0,20,loam
3,0,50,50,100,0,20,silt loam
4,50,80,0,30,20,30,sandy clay loam


### Assign soil texture type based on soil lookup table

In [6]:
textures = []
for i in np.arange(df_soils.shape[0]):
    
    sand = df_soils.loc[i,'sand']
    silt = df_soils.loc[i,'silt']
    clay = df_soils.loc[i,'clay']

    texture = df_texture[
        (sand > df_texture.sand_min) & (sand <= df_texture.sand_max) & 
        (silt > df_texture.silt_min) & (silt <= df_texture.silt_max) & 
        (clay > df_texture.clay_min) & (clay <= df_texture.clay_max)].texture.str.cat()
    textures.append(texture)

df_soils['texture'] = textures

### Group soil texture into broader soil groups

In [7]:
# create column for soil group
df_soils['sgroup'] = ''

# soil group 1: sand
soil_category = ((df_soils.texture == 'sand') | (df_soils.texture == 'sandy loam') | 
                 (df_soils.texture == 'sandy clay loam') | (df_soils.texture == 'sandy clay'))
df_soils.loc[soil_category, 'sgroup'] = 1

# soil group 2: loam
soil_category = ((df_soils.texture == 'clay loam') | (df_soils.texture == 'loam'))
df_soils.loc[soil_category, 'sgroup'] = 2

# soil group 3: silt 
soil_category = ((df_soils.texture == 'silty clay') | (df_soils.texture == 'silty clay loam') | 
                 (df_soils.texture == 'silt loam'))
df_soils.loc[soil_category, 'sgroup'] = 3

# soil group 4: clay
soil_category = ((df_soils.texture == 'clay'))
df_soils.loc[soil_category, 'sgroup'] = 4

In [8]:
df_soils.head()

,cokey,chkey,prcent,slope_r,slope,hzname,depth,awc,clay,silt,...,OM,dbthirdbar,th33,bd,site,lat,lon,depth_category,texture,sgroup
0,20681154,60804747,60.0,1,2,H1,0.0,0.07,7.0,6.6,...,1.50,1.55,16.6,-0.1505,722140,32.117,-81.20,0,sand,1
1,20681154,60804748,60.0,1,2,H2,71.0,0.11,25.0,18.0,...,0.25,1.63,24.9,-0.2327,722140,32.117,-81.20,50-100,sandy clay loam,1
2,20681154,60804749,60.0,1,2,H3,150.0,0.11,27.5,17.4,...,0.25,1.63,25.9,-0.2427,722140,32.117,-81.20,100-150,sandy clay loam,1
3,20646330,60671139,85.0,3,5,A,0.0,0.04,1.5,4.9,...,0.53,1.45,5.2,-0.0375,725330,30.400,-84.35,0,sand,1
4,20646330,60671138,85.0,3,5,C,18.0,0.04,1.0,1.1,...,0.05,1.45,4.0,-0.0255,725330,30.400,-84.35,0-50,sand,1


### Output soil info

In [9]:
#df_soils.to_csv('/home/disk/eos8/ach315/upscale/weadata/soiltexture.csv')